In [77]:
import pandas as pd
import numpy as np
import holidays
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline

In [4]:
df=pd.read_csv('./data/Sample Dataset.csv')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39408 entries, 0 to 39407
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Date               39408 non-null  object 
 1   Weekday            39408 non-null  object 
 2   Hour               39408 non-null  int64  
 3   HOEP               39408 non-null  float64
 4   Ontario_Demand     39408 non-null  int64  
 5   Temperature        39408 non-null  float64
 6   Windchill_Index    39408 non-null  float64
 7   Wind_Speed         39408 non-null  int64  
 8   Humidex            39408 non-null  float64
 9   Relative_Humidity  39408 non-null  int64  
 10  Dew_Point          39408 non-null  float64
 11  Pressure_Station   39408 non-null  float64
dtypes: float64(6), int64(4), object(2)
memory usage: 3.6+ MB


In [9]:
df['Hour']=df['Hour'].astype(str)

In [11]:
df['Hour'] = df['Hour'].apply(lambda x: '%s:00:00' % x)

In [13]:
df.head()

,Date,Weekday,Hour,HOEP,Ontario_Demand,Temperature,Windchill_Index,Wind_Speed,Humidex,Relative_Humidity,Dew_Point,Pressure_Station
0,2016-01-01,Friday,0:00:00,0.49,14023,-0.3,-5.62,20,-3.18,70,-5.1,99.67
1,2016-01-01,Friday,1:00:00,-1.09,13417,-0.3,-6.30,25,-3.48,68,-5.5,99.63
2,2016-01-01,Friday,2:00:00,-2.41,12968,-0.4,-6.55,26,-3.43,73,-4.7,99.59
3,2016-01-01,Friday,12:00:00,0.00,14215,-2.1,-8.72,23,-4.86,76,-5.8,99.39
4,2016-01-01,Friday,13:00:00,0.00,14443,-2.4,-8.23,19,-5.09,84,-4.7,99.35


In [15]:
df_orig = df.copy()

In [17]:
# Highlighting NaNs and reshaping dataframe
df_start = df[['Date', 'Weekday']].drop_duplicates()
df_end = df_start.copy()
df_start['Hour'] = '00:00:00'
df_end['Hour'] = '23:00:00'
df_comb = pd.concat([df_start,df_end])

# Datetime column (drop Hour columns)
df['Datetime'] = pd.to_datetime(df.Date + ' ' + df.Hour)
df.drop('Hour', inplace=True, axis = 1)
df_comb['Datetime'] = pd.to_datetime(df_comb.Date + ' ' + df_comb.Hour)
df_comb.drop('Hour', inplace=True, axis = 1)

# Add NaNs
df = pd.concat([df, df_comb],  ignore_index=True)\
    .drop_duplicates(subset=['Weekday', 'Datetime'])

# Create the complete set of readings
df = df.groupby(['Date', 'Weekday'])\
    .apply(lambda x : x.set_index('Datetime').resample('h').mean().fillna(np.NaN))\
    .reset_index()
df.drop('Date', inplace=True, axis = 1)


In [18]:
df.head(49)

,Weekday,Datetime,HOEP,Ontario_Demand,Temperature,Windchill_Index,Wind_Speed,Humidex,Relative_Humidity,Dew_Point,Pressure_Station
0,Friday,2016-01-01 00:00:00,0.49,14023.0,-0.3,-5.62,20.0,-3.18,70.0,-5.1,99.67
1,Friday,2016-01-01 01:00:00,-1.09,13417.0,-0.3,-6.30,25.0,-3.48,68.0,-5.5,99.63
2,Friday,2016-01-01 02:00:00,-2.41,12968.0,-0.4,-6.55,26.0,-3.43,73.0,-4.7,99.59
3,Friday,2016-01-01 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Friday,2016-01-01 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Friday,2016-01-01 05:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Friday,2016-01-01 06:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Friday,2016-01-01 07:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Friday,2016-01-01 08:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Friday,2016-01-01 09:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
df_orig.head(49)

,Date,Weekday,Hour,HOEP,Ontario_Demand,Temperature,Windchill_Index,Wind_Speed,Humidex,Relative_Humidity,Dew_Point,Pressure_Station
0,2016-01-01,Friday,0:00:00,0.49,14023,-0.3,-5.62,20,-3.18,70,-5.1,99.67
1,2016-01-01,Friday,1:00:00,-1.09,13417,-0.3,-6.30,25,-3.48,68,-5.5,99.63
2,2016-01-01,Friday,2:00:00,-2.41,12968,-0.4,-6.55,26,-3.43,73,-4.7,99.59
3,2016-01-01,Friday,12:00:00,0.00,14215,-2.1,-8.72,23,-4.86,76,-5.8,99.39
4,2016-01-01,Friday,13:00:00,0.00,14443,-2.4,-8.23,19,-5.09,84,-4.7,99.35
5,2016-01-01,Friday,14:00:00,0.00,14733,-2.4,-9.96,32,-5.42,81,-5.2,99.32
6,2016-01-01,Friday,15:00:00,0.00,14907,-2.2,-9.75,29,-5.59,71,-6.8,99.34
7,2016-01-01,Friday,16:00:00,0.00,14981,-2.3,-9.51,27,-5.79,68,-7.4,99.38
8,2016-01-01,Friday,17:00:00,1.00,15763,-3.1,-11.19,35,-5.82,71,-7.6,99.41
9,2016-01-01,Friday,18:00:00,6.34,16979,-2.9,-11.19,35,-6.39,71,-7.4,99.48


In [23]:
ont_holidays=holidays.country_holidays('CA', 'ON')

In [25]:
ont_holidays

holidays.country_holidays('CA', subdiv='ON')

In [27]:
df.Datetime.dt.date

0        2016-01-01
1        2016-01-01
2        2016-01-01
3        2016-01-01
4        2016-01-01
            ...    
43843    2020-12-31
43844    2020-12-31
43845    2020-12-31
43846    2020-12-31
43847    2020-12-31
Name: Datetime, Length: 43848, dtype: object

In [29]:
df['is_holiday'] = df.Datetime.dt.date.apply(lambda x: 1 if x in ont_holidays else 0)

In [31]:
categorical_columns = df.select_dtypes(include=['object']).columns.tolist()

In [33]:
categorical_columns

['Weekday']

In [35]:
encoder = OneHotEncoder(sparse_output=False)

# Apply one-hot encoding to the categorical columns
one_hot_encoded = encoder.fit_transform(df[categorical_columns])

#Create a DataFrame with the one-hot encoded columns
#Use get_feature_names_out() to get the column names for the encoded data
one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(categorical_columns))

# Concatenate the one-hot encoded dataframe with the original dataframe
df_encoded = pd.concat([df, one_hot_df], axis=1)

# Drop the original categorical columns
df_encoded = df_encoded.drop('Weekday', axis=1)

In [37]:
df_encoded.head()

,Datetime,HOEP,Ontario_Demand,Temperature,Windchill_Index,Wind_Speed,Humidex,Relative_Humidity,Dew_Point,Pressure_Station,is_holiday,Weekday_Friday,Weekday_Monday,Weekday_Saturday,Weekday_Sunday,Weekday_Thursday,Weekday_Tuesday,Weekday_Wednesday
0,2016-01-01 00:00:00,0.49,14023.0,-0.3,-5.62,20.0,-3.18,70.0,-5.1,99.67,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2016-01-01 01:00:00,-1.09,13417.0,-0.3,-6.30,25.0,-3.48,68.0,-5.5,99.63,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2016-01-01 02:00:00,-2.41,12968.0,-0.4,-6.55,26.0,-3.43,73.0,-4.7,99.59,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2016-01-01 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2016-01-01 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0


## filling missing values

In [40]:
df_encoded.shape

(43848, 18)

In [42]:
df_encoded.isnull().sum()

Datetime                0
HOEP                 4443
Ontario_Demand       4443
Temperature          4443
Windchill_Index      4443
Wind_Speed           4443
Humidex              4443
Relative_Humidity    4443
Dew_Point            4443
Pressure_Station     4443
is_holiday              0
Weekday_Friday          0
Weekday_Monday          0
Weekday_Saturday        0
Weekday_Sunday          0
Weekday_Thursday        0
Weekday_Tuesday         0
Weekday_Wednesday       0
dtype: int64

10% missing values 

let use a linear interpolation function for filling missing values

In [45]:
df_encoded.set_index('Datetime', inplace=True)

In [47]:
#for later checking whether nans are replaced 
nan_demand = df_encoded[df_encoded['Ontario_Demand'].isnull()==True].index

nan_demand

DatetimeIndex(['2016-01-01 03:00:00', '2016-01-01 04:00:00',
               '2016-01-01 05:00:00', '2016-01-01 06:00:00',
               '2016-01-01 07:00:00', '2016-01-01 08:00:00',
               '2016-01-01 09:00:00', '2016-01-01 10:00:00',
               '2016-01-01 11:00:00', '2016-01-02 04:00:00',
               ...
               '2020-12-10 10:00:00', '2020-12-17 08:00:00',
               '2020-12-17 13:00:00', '2020-12-17 18:00:00',
               '2020-12-17 19:00:00', '2020-12-17 21:00:00',
               '2020-12-17 22:00:00', '2020-12-18 08:00:00',
               '2020-12-18 09:00:00', '2020-12-31 18:00:00'],
              dtype='datetime64[ns]', name='Datetime', length=4443, freq=None)

In [49]:
df_final = df_encoded.interpolate(method='linear', axis=0)

In [51]:
duplicated_rows_bool =df_final.index.duplicated()

In [53]:
sum(duplicated_rows_bool)

0

In [55]:
df_final.isnull().sum()

HOEP                 0
Ontario_Demand       0
Temperature          0
Windchill_Index      0
Wind_Speed           0
Humidex              0
Relative_Humidity    0
Dew_Point            0
Pressure_Station     0
is_holiday           0
Weekday_Friday       0
Weekday_Monday       0
Weekday_Saturday     0
Weekday_Sunday       0
Weekday_Thursday     0
Weekday_Tuesday      0
Weekday_Wednesday    0
dtype: int64

In [57]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 43848 entries, 2016-01-01 00:00:00 to 2020-12-31 23:00:00
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   HOEP               43848 non-null  float64
 1   Ontario_Demand     43848 non-null  float64
 2   Temperature        43848 non-null  float64
 3   Windchill_Index    43848 non-null  float64
 4   Wind_Speed         43848 non-null  float64
 5   Humidex            43848 non-null  float64
 6   Relative_Humidity  43848 non-null  float64
 7   Dew_Point          43848 non-null  float64
 8   Pressure_Station   43848 non-null  float64
 9   is_holiday         43848 non-null  int64  
 10  Weekday_Friday     43848 non-null  float64
 11  Weekday_Monday     43848 non-null  float64
 12  Weekday_Saturday   43848 non-null  float64
 13  Weekday_Sunday     43848 non-null  float64
 14  Weekday_Thursday   43848 non-null  float64
 15  Weekday_Tuesday    43848 non-null  

In [59]:
#look at the interapolated values 
df_interpolate=df_final[df_final.index.isin(nan_demand)]

In [61]:
df_interpolate.head(6)

,HOEP,Ontario_Demand,Temperature,Windchill_Index,Wind_Speed,Humidex,Relative_Humidity,Dew_Point,Pressure_Station,is_holiday,Weekday_Friday,Weekday_Monday,Weekday_Saturday,Weekday_Sunday,Weekday_Thursday,Weekday_Tuesday,Weekday_Wednesday
Datetime,,,,,,,,,,,,,,,,,
2016-01-01 03:00:00,-2.169,13092.7,-0.57,-6.767,25.7,-3.573,73.3,-4.81,99.57,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-01 04:00:00,-1.928,13217.4,-0.74,-6.984,25.4,-3.716,73.6,-4.92,99.55,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-01 05:00:00,-1.687,13342.1,-0.91,-7.201,25.1,-3.859,73.9,-5.03,99.53,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-01 06:00:00,-1.446,13466.8,-1.08,-7.418,24.8,-4.002,74.2,-5.14,99.51,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-01 07:00:00,-1.205,13591.5,-1.25,-7.635,24.5,-4.145,74.5,-5.25,99.49,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-01 08:00:00,-0.964,13716.2,-1.42,-7.852,24.2,-4.288,74.8,-5.36,99.47,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [63]:
df_final.head(6)

,HOEP,Ontario_Demand,Temperature,Windchill_Index,Wind_Speed,Humidex,Relative_Humidity,Dew_Point,Pressure_Station,is_holiday,Weekday_Friday,Weekday_Monday,Weekday_Saturday,Weekday_Sunday,Weekday_Thursday,Weekday_Tuesday,Weekday_Wednesday
Datetime,,,,,,,,,,,,,,,,,
2016-01-01 00:00:00,0.490,14023.0,-0.30,-5.620,20.0,-3.180,70.0,-5.10,99.67,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-01 01:00:00,-1.090,13417.0,-0.30,-6.300,25.0,-3.480,68.0,-5.50,99.63,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-01 02:00:00,-2.410,12968.0,-0.40,-6.550,26.0,-3.430,73.0,-4.70,99.59,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-01 03:00:00,-2.169,13092.7,-0.57,-6.767,25.7,-3.573,73.3,-4.81,99.57,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-01 04:00:00,-1.928,13217.4,-0.74,-6.984,25.4,-3.716,73.6,-4.92,99.55,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-01 05:00:00,-1.687,13342.1,-0.91,-7.201,25.1,-3.859,73.9,-5.03,99.53,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
column = df_final.pop('Ontario_Demand') 
 
df_final.insert(0, 'Ontario_Demand', column)

In [67]:
df_final.head()

,Ontario_Demand,HOEP,Temperature,Windchill_Index,Wind_Speed,Humidex,Relative_Humidity,Dew_Point,Pressure_Station,is_holiday,Weekday_Friday,Weekday_Monday,Weekday_Saturday,Weekday_Sunday,Weekday_Thursday,Weekday_Tuesday,Weekday_Wednesday
Datetime,,,,,,,,,,,,,,,,,
2016-01-01 00:00:00,14023.0,0.490,-0.30,-5.620,20.0,-3.180,70.0,-5.10,99.67,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-01 01:00:00,13417.0,-1.090,-0.30,-6.300,25.0,-3.480,68.0,-5.50,99.63,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-01 02:00:00,12968.0,-2.410,-0.40,-6.550,26.0,-3.430,73.0,-4.70,99.59,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-01 03:00:00,13092.7,-2.169,-0.57,-6.767,25.7,-3.573,73.3,-4.81,99.57,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-01 04:00:00,13217.4,-1.928,-0.74,-6.984,25.4,-3.716,73.6,-4.92,99.55,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [79]:
## apply PCA pipeline to the set of features related to the weather.

#initate a min max scaler
minmax = MinMaxScaler()

#initate a PCA object 
pca = PCA(n_components=2)

#initate the pipeline for pca
pca_pipe = make_pipeline(pca, minmax)

sel_cols = ['HOEP','Temperature','Windchill_Index','Wind_Speed','Humidex','Relative_Humidity','Dew_Point','Pressure_Station']

pca_sel = pca_pipe.fit_transform(df_final[sel_cols])

#make dataframe with the transformed weather pca
pca_sel_df = pd.DataFrame(pca_sel, index=df_final.index, columns=['pca1', 'pca2'])

In [81]:
pca_sel_df.shape

(43848, 2)

In [83]:
df_final.shape

(43848, 17)

In [85]:
df_pca = df_final.drop(sel_cols, axis=1)

In [87]:
df_pca.head(1)

,Ontario_Demand,is_holiday,Weekday_Friday,Weekday_Monday,Weekday_Saturday,Weekday_Sunday,Weekday_Thursday,Weekday_Tuesday,Weekday_Wednesday
Datetime,,,,,,,,,
2016-01-01,14023.0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [89]:
df_pca.insert(1, "pca1", pca_sel_df['pca1'])

In [91]:
df_pca.head(1)

,Ontario_Demand,pca1,is_holiday,Weekday_Friday,Weekday_Monday,Weekday_Saturday,Weekday_Sunday,Weekday_Thursday,Weekday_Tuesday,Weekday_Wednesday
Datetime,,,,,,,,,,
2016-01-01,14023.0,0.592382,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [93]:
df_pca.insert(2, "pca2", pca_sel_df['pca2'])

In [95]:
df_pca.head()

,Ontario_Demand,pca1,pca2,is_holiday,Weekday_Friday,Weekday_Monday,Weekday_Saturday,Weekday_Sunday,Weekday_Thursday,Weekday_Tuesday,Weekday_Wednesday
Datetime,,,,,,,,,,,
2016-01-01 00:00:00,14023.0,0.592382,0.395017,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-01 01:00:00,13417.0,0.598022,0.423531,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-01 02:00:00,12968.0,0.601944,0.370056,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-01 03:00:00,13092.7,0.604428,0.367016,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-01 04:00:00,13217.4,0.606912,0.363976,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [97]:
df_pca.to_csv('./data/clean_data_ondemand_pca.csv')